Activation of Python API

In [9]:
from PythonAPI import *
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

Load data

In [64]:
df_train = pd.read_csv('../data1/train_rf.csv')
df_train = df_train.iloc[:, 1:]

df_test = pd.read_csv('../data1/test_rf.csv')
df_test = df_test.iloc[:, 1:]

Scale data

In [65]:
def scale_data(df):
    scaler = MinMaxScaler()
    df.iloc[:, :-1] = scaler.fit_transform(df.iloc[:, :-1])
    return df

In [66]:
df_train = scale_data(df_train)
df_test = scale_data(df_test)

/var/folders/rh/w6cl6rd95vj2g4s03scz_4fc0000gn/T/ipykernel_59423/3975466352.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, :-1] = scaler.fit_transform(df.iloc[:, :-1])
/var/folders/rh/w6cl6rd95vj2g4s03scz_4fc0000gn/T/ipykernel_59423/3975466352.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, :-1] = scaler.fit_transform(df.iloc[:, :-1])


In [69]:
df_train.to_csv('../data1/train_rf_scaled.csv', index=False)
df_test.to_csv('../data1/test_rf_scaled.csv', index=False)

In [70]:
x_names = df_train.columns[:-1]
y_name = df_train.columns[-1]
x_names

Index(['Default_flag_H0_H3', 'utilized_limit_in_revolving_loans_H0',
       'Default_flag_H0_H6', 'DPD_term_loan_H0_H3', 'Os_term_loan_H0_H3',
       'Payments_term_loan_H0_H3', 'Os_term_loan_H0_H6',
       'Payments_term_loan_H0_H6', 'Income_H0'],
      dtype='object')

We are configuring the username and password. Each team possesses a distinct pair of credentials

In [71]:
user = "team02"
password = "lQys9Nt]%n@v"

We are verifying our ability to establish a connection with the finQbit API. If the connection is successful, the method will return "OK".
Additional details regarding the method can be found in Chapter 4.1 of the API documentation.

In [72]:
check_connection()

'OK'

Subsequently, you should transfer the training data, which has been prepared with up to 9 features per item represented as numbers within the 0-1 range, through FTP. Utilize the identical username and password for FTP access that are used for API connections.

After transferring a file to FTP, we must notify the system so that it can load and preprocess the file accordingly. This is achieved using the loadData method. Additional details regarding the method can be found in Chapter 4.3 of the API documentation.

In [73]:
load_data(user, password, 'train_rf_scaled.csv',
         ['Default_flag_H0_H3', 'utilized_limit_in_revolving_loans_H0',
       'Default_flag_H0_H6', 'DPD_term_loan_H0_H3', 'Os_term_loan_H0_H3',
       'Payments_term_loan_H0_H3', 'Os_term_loan_H0_H6',
       'Payments_term_loan_H0_H6', 'Income_H0'],
         'Target')
# load_data(user, password, "trainDataBig.csv",
#       ["Dependents", "Education", "Self_Employed", "ApplicantIncome", "CoapplicantIncome", "LoanAmount", "Loan_Amount_Term", "Credit_History", "Property_Area"],
#       "Loan_Status")    

'OK'

Given that processing occurs in a separate thread, the server's response merely confirms that the data has been accepted for processing. 
Therefore, one must monitor the status of the processing. We anticipate the message "The data was loaded correctly."
Additional details regarding the method can be found in Chapter 4.2 of the API documentation.

In [74]:
check_status(user, password)

'Pre-process of data.'

Following that, we start the training process, for which parameter details are available in Chapter 4.4 of the API documentation.
It's important to verify the training progress by using the "check_status" method, looking for the confirmation: "The model was trained correctly."

In [77]:
train(user, password, 100, 100, 0.1, "Eva", "SU2U4")

'OK'

In [80]:
check_status(user, password)

'team02: New best parameters found: loss=0.9617429733276366'

We retrieve the identified optimization parameters. Additional details regarding the method can be found in Chapter 4.5 of the API documentation.

In [81]:
params = get_parameters(user, password)

Parameters are specific to each network structure and feature count combination. However, these combinations can undergo training with various optimization techniques.
Providing these parameters allows us to resume optimization from where it was last left off.

In [97]:
# tutaj podane parametry ktore znalazlem
train(user, password, 100, 100, 0.1, "Adam", "SU2U4", params=params,
      beta=0.1, beta2=0.1)

'OK'

In [99]:
check_status(user, password)

'team02: New best parameters found: loss=0.8556270790100098'

In [88]:
params = get_parameters(user, password)

Once our model is trained, we can evaluate its performance with a specified initial vector by utilizing the score method.
Additional details regarding the method can be found in Chapter 4.6 of the API documentation.

In [90]:
score(user, password, "SU2U4", [0.2222222222222224,0.41666666666666663,0.09756097560975609,0.05882352941176471], params)

'ERROR: The number of model parameters disagrees with the number of provided parameters. Check if you use the same model with the same number of input features.'

Invoking the score function on the test dataset directly would not be efficient. Hence, it's advisable to transfer the test dataset to the same location as the training dataset using FTP.
This allows for validation across the entire dataset. Details about the method's parameters are available in the API documentation in Chapter 4.6.

In [93]:
# validate(user, password, "testIris.csv",
#          ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"],
#          "Species", "SU2U4", params)
validate(user, password, "test_rf_scaled.csv",
         ['Default_flag_H0_H3', 'utilized_limit_in_revolving_loans_H0',
       'Default_flag_H0_H6', 'DPD_term_loan_H0_H3', 'Os_term_loan_H0_H3',
       'Payments_term_loan_H0_H3', 'Os_term_loan_H0_H6',
       'Payments_term_loan_H0_H6', 'Income_H0'],
         "Target", "SU2U4", params)  

'OK'

Keep in mind that the validation process runs on a separate thread, with the method returning confirmation that the server has undertaken the task.
To ascertain the outcome of the validation, one must obtain the status.

In [96]:
check_status(user, password)

'Accuracy=0.6059093516924843, precision=0.14679062583538777, recall=0.6933707471062786, AUC=0.7072733286076687'